In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    #time.sleep(0.5)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    #time.sleep(0.5)    
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    #time.sleep(0.5)
    #driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(2)
    sp = soup(driver.page_source, 'lxml')

# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'lxml')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'lxml')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

In [2]:
urls = pd.read_csv('mlb_stats_urls_2022.csv', encoding='utf-8')

for year in range(2022, 2023):
    for link in urls['link']:
        df_list = []
        for page in range(1, 4):
            url = link + str(year) + '?page=' + str(page) + '&playerPool=ALL'
            driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
            html = driver.get(url)
            sp = soup(driver.page_source, 'lxml')
            column_name = get_column_name()
            try:
                #while True:
                sp = soup(driver.page_source, 'lxml')
                print('page start')
                time.sleep(1)
                name = get_player_name()
                time.sleep(1)
                position = get_player_position()
                time.sleep(1)
                current_data = get_data()
                time.sleep(1)
                for i in range(len(name)):
                    row = []
                    row.append(name[i])
                    row.append(position[i])
                    row.extend(current_data[i*32:i*32+32])
                    df_list.append(row)
                time.sleep(1)
                #click_next_page_button()
            except:
                print('This is the last page :)')
        df = pd.DataFrame(df_list, columns=column_name)
        df.to_csv(f'mlb_stats/{year}/{link[26:-1]}_{year}.csv', index=False, encoding='utf-8')
        print(df)

C:\Users\user\AppData\Local\Temp\ipykernel_44360\4169385497.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
              PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0         Mike Trout       CF  LAA  119  438  85  124  28  2  40  ...  0.43   
1         Livan Soto       SS  LAA   18   55   9   22   5  1   1  ...  0.83   
2      Shohei Ohtani       DH  LAA  157  586  90  160  30  6  34  ...  1.03   
3        Taylor Ward       RF  LAA  135  495  73  139  22  2  23  ...  0.64   
4     Monte Harrison       LF  LAA    9   11   5    2   0  0   1  ...  1.00   
5       Luis Rengifo       2B  LAA  127  489  45  129  22  4  17  ...  1.10   
6     Anthony Rendon       3B  LAA   47  166  15   38  10  0   5  ...  0.67   
7          Mike Ford       1B  LAA   28   91   8   21   4  0   3  ...  0.42   
8      Logan O'Hoppe        C  LAA    5   14   1    4   0  0   0  ...  0.40   
9      Mickey Moniak       LF  LAA   19   60   9   12   2  1   3  ...  0.35   
10       Jared Walsh       1B  LAA  118  423  41   91  18  2  15  ...  1.11   
11       Matt Thais

C:\Users\user\AppData\Local\Temp\ipykernel_44360\4169385497.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ... GO/AO  \
0       David Hensley       DH  HOU   16   29    7   10   2  1   1  ...     1   
1      Yordan Alvarez       DH  HOU  135  470   95  144  29  2  37  ...    29   
2         Jose Altuve       2B  HOU  141  527  103  158  39  0  28  ...   158   
3        Alex Bregman       3B  HOU  155  548   93  142  38  0  23  ...    93   
4         Kyle Tucker       RF  HOU  150  544   71  140  28  1  30  ...   544   
5    Michael Brantley       DH  HOU   64  243   28   70  14  1   5  ...    64   
6      Chas McCormick       LF  HOU  119  359   47   88  12  2  14  ...   HOU   
7         Jeremy Pena       SS  HOU  136  521   72  132  20  2  22  ...  .739   
8        Aledmys Diaz       LF  HOU   92  305   35   74  13  0  12  ...  .426   
9        Yuli Gurriel       1B  HOU  146  545   53  132  40  0   8  ...  .287   
10       Trey Mancini       DH  HOU   51  165   17   29   7  0   8  ...  .24

C:\Users\user\AppData\Local\Temp\ipykernel_44360\4169385497.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


...

page start
page start
page start
                PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ... GO/AO  \
0          Royce Lewis       SS  MIN   12   40   5   12   4  0   2  ...  0.53   
1        Carlos Correa       SS  MIN  136  522  70  152  24  1  22  ...  0.98   
2         Byron Buxton       CF  MIN   92  340  61   76  13  3  28  ...  0.60   
3          Luis Arraez       1B  MIN  144  547  88  173  31  1   8  ...  0.92   
4          Gio Urshela       3B  MIN  144  501  61  143  27  3  13  ...  0.82   
5         Jose Miranda       1B  MIN  125  444  45  119  25  0  15  ...  0.88   
6        Jorge Polanco       2B  MIN  104  375  54   88  16  0  16  ...  0.56   
7          Nick Gordon       LF  MIN  138  405  45  110  28  4   9  ...  0.99   
8         Kyle Garlick       LF  MIN   66  150  23   35   3  0   9  ...  1.00   
9       Trevor Larnach       LF  MIN   51  160  22   37  13  0   5  ...  1.13   
10        Matt Wallner       RF  MIN   18   57   4   13   3  0   2  ...  1.3

C:\Users\user\AppData\Local\Temp\ipykernel_44360\4169385497.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
               PLAYER POSITION TEAM    G   AB   R    H  2B 3B  HR  ...  GO/AO  \
0        Eloy Jimenez       DH  CWS   84  292  40   86  12  0  16  ...   1.30   
1          Jose Abreu       1B  CWS  157  601  85  183  40  0  15  ...   1.21   
2       Danny Mendick       SS  CWS   31   97  22   28   4  1   3  ...   0.96   
3        Elvis Andrus       SS  CWS   43  181  25   49   8  0   9  ...   1.43   
4         Jake Burger       3B  CWS   51  168  20   42   9  1   8  ...   0.89   
5       Andrew Vaughn       RF  CWS  134  510  60  138  28  1  17  ...   1.18   
6         Luis Robert       CF  CWS   98  380  54  108  18  0  12  ...   0.98   
7        Tim Anderson       SS  CWS   79  332  50  100  13  0   6  ...   1.81   
8         Seby Zavala        C  CWS   61  178  22   48  14  0   2  ...   0.48   
9        Gavin Sheets       RF  CWS  124  377  34   91  19  0  15  ...   0.74   
10      Josh Harrison       2B  CWS  119  386  50   99  19  2   7  ...   0.9

C:\Users\user\AppData\Local\Temp\ipykernel_44360\4169385497.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


page start
page start
page start
                PLAYER POSITION TEAM    G   AB    R    H  2B 3B  HR  ...  \
0       Matt Carpenter       DH  NYY   47  128   28   39   9  0  15  ...   
1          Aaron Judge       CF  NYY  157  570  133  177  28  0  62  ...   
2          Rob Brantly        C  NYY    1    3    0    1   1  0   0  ...   
3        Oswald Peraza       SS  NYY   18   49    8   15   3  0   1  ...   
4        Anthony Rizzo       1B  NYY  130  465   77  104  21  1  32  ...   
5       Gleyber Torres       2B  NYY  140  526   73  135  28  1  24  ...   
6    Giancarlo Stanton       DH  NYY  110  398   53   84   7  0  31  ...   
7      Oswaldo Cabrera       RF  NYY   44  154   21   38   8  1   6  ...   
8    Andrew Benintendi       LF  NYY   33  114   14   29   9  1   2  ...   
9          DJ LeMahieu       3B  NYY  125  467   74  122  18  0  12  ...   
10      Josh Donaldson       3B  NYY  132  478   59  106  28  0  15  ...   
11        Jose Trevino        C  NYY  115  335   39   8

In [ ]:
driver.quit()